In [ ]:
import csv
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
import smtplib
import getpass
from reportlab.pdfgen import canvas
from email import encoders

def create_pdf(name, ticket_number, province):
    pdf = canvas.Canvas('tiket_seminar.pdf')
    title = 'Tiket Webinar FKMBU 2021'
    text1 = 'Hai {}'
    text2 = 'Berikut kami lampirkan tiket Webinar FKMBU 2021 beserta data pembeli, Terima kasih'
    text3 = 'Nama: {}'
    text4 = 'Provinsi: {}'
    text5 = 'Nomor Tiket: {}'
    footer1 = 'Hormat Kami,'
    footer2 = 'Panitia Pelaksana'
    image = ('Ticket.JPG')
    pdf.drawString(230, 750, title)
    pdf.drawString(72, 700, text1.format(name))
    pdf.drawString(72, 680, text2)
    pdf.drawString(72, 660, text3.format(name))
    pdf.drawString(72, 640, text4.format(province))
    pdf.drawString(72, 620, text5.format(ticket_number))
    pdf.drawInlineImage(image, 72, 430, 470, 170)
    pdf.drawString(72, 170, footer1)
    pdf.drawString(72, 100, footer2)
    pdf.save()

data = open('list.csv')
csv_data = csv.reader(data)
data_lines = list(csv_data)
names = []
emails = []
tickets = []
provinsi = []

for item in data_lines[1:]:
    names.append(item[2])
    emails.append(item[1])
    tickets.append(item[4])
    provinsi.append(item[3])
    
    
subject = 'Tiket Webinar FKMBU 2021'
body = '''
Kepada Yth.
{}

Terima kasih sudah melakukan registrasi pada acara Webinar Nasional FKMBU 2021 dengan tema “Ketahuilah Potensimu dan Kembangkan Bakatmu” yang diselenggarakan pada:

Hari/tanggal: Sabtu, 17 April 2021
Waktu: 09.00 WITA-Selesai
Link Meeting: Akan diumumkan di grup Line Square

Peserta diharapkan untuk bergabung ke Line Square dengan format nama NomorTiket_Nama, membaca tata tertib, mengunduh, dan menggunakan background yang telah kami sediakan melalui link dibawah ini:

Link Line Square: https://line.me/ti/g2/srusKp-oL1ku_Ac-JKlNCQ?utm_source=invitation&utm_medium=link_copy&utm_campaign=default
Link Folder Google Drive: https://drive.google.com/drive/folders/1dRusZV-mjvlyWX2opmgtKDslNKNZK1RD?usp=sharing

Jika anda memiliki pertanyaan silahkan hubungi contact person dibawah ini:
#: # (ID Line = #)
#: # (ID Line = #)

Sekian informasi yang dapat kami sampaikan. Terima kasih dan selamat mengikuti Webinar FKMBU 2021!

Hormat kami,



Panitia Webinar FKMBU 2021

'''

smtp_object = smtplib.SMTP('smtp.gmail.com', 587)
smtp_object.starttls()
email = getpass.getpass("Enter your email: ")
password = getpass.getpass("Enter your password: ")
smtp_object.login(email,password)
counter = 0

for name in names:
    msg = MIMEMultipart()
    msg.attach(MIMEText(body.format(name), 'plain'))
    create_pdf(name, tickets[counter], provinsi[counter])
    file_name = 'tiket_seminar.pdf'
    attachment = open(file_name, 'rb')
    part = MIMEBase('application', 'octet-stream')
    part.set_payload((attachment).read())
    part.add_header('Content-Disposition', 'attachment; filename= ' +file_name)
    encoders.encode_base64(part)
    msg.attach(part)
    msg['Subject'] = subject
    msg['From'] = email
    msg['To'] = emails[counter]
    smtp_object.sendmail(email, emails[counter], msg.as_string())
    counter += 1
    attachment.close()
    
smtp_object.close()